In [5]:
import os
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

intents_path = '/content/drive/MyDrive/Colab Notebooks/chatbot-flask-simple/data/intents'

# inference model variables
inference_load_intents_from = os.path.join(intents_path, 'intents_job_intents.json')

words = []
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open(inference_load_intents_from, encoding='cp1252').read()
intents = json.loads(data_file)

for intent in intents['intents']:
     for pattern in intent['patterns']:
          w = nltk.word_tokenize(pattern)
          words.extend(w)

          documents.append((w, intent['tag']))

          if intent['tag'] not in classes:
               classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print(len(documents), 'documents')
print(len(classes), 'classes', classes)
print(len(words), 'unique lemmatized words', words)

pickle.dump(words,open(os.path.join(intents_path, 'intents_words.pkl'),'wb'))
pickle.dump(classes,open(os.path.join(intents_path, 'intents_classes.pkl'),'wb'))

# init training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
     bag = []
     pattern_words = doc[0]
     pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

     for w in words:
          bag.append(1) if w in pattern_words else bag.append(0)

     output_row = list(output_empty)
     output_row[classes.index(doc[1])] = 1

     training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)
# create train and test lists.  X - patterns, y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print('Training data created')
print(f'Train_x: {train_x}')
print(f'Train_y: {train_y}')
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, 
                                                    test_size=0.25, 
                                                    random_state=1)
print(f'X_train: {X_train}')
print(f'X_train.shape: {len(X_train)}')
print(f'y_train: {y_train}')
print(f'y_train.shape: {len(y_train)}')
print(f' X_test:  {X_test}')
print(f' X_test.shape: {len(X_test)}')
print(f' y_test:  {y_test}')
print(f' y_test.shape: {len(y_test)}')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
61 documents
11 classes ['compliment', 'feeling', 'goodbye', 'greeting', 'manager', 'name', 'package_tracking', 'profane', 'return_product', 'thanks', 'tracking']
88 unique lemmatized words ["'", "'s", ',', '.', '0983834298342341', '1234509873234323', '1983-2343-2343-2343', 'a', 'am', 'are', 'back', 'broken', 'bye', 'can', 'computer', 'dead', 'device', 'die', 'do', 'doing', 'drop', 'equipment', 'feel', 'feeling', 'fix', 'fixed', 'get', 'give', 'go', 'going', 'good', 'goodbye', 'hate', 'have', 'hear', 'hello', 'help', 'helpful', 'here', 'hi', 'how', 'i', 'is', 'it', 'later', 'like', 'long', 'manager', 'me', 'money', 'my', 'name', 'need', 'not', 'now', 'package', 'product', 'refund', 'report', 'return', 'see', 'shipment', 'some', 'something', 'speak', 'speaking', 's

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:67: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [10]:


# Create model with 3 layers.  First layer 128 neurons, second layer 64 neurons
# and 3rd output layer contains number of neurons equal to number of intents to
# predict
# output intent with softmax
model = Sequential()
model.add(Embedding
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model.  Stochastic gradient descent with Nesterov accelerated
# gradient gives good
# results for this model
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# fitting and saving the model
hist = model.fit(np.array(X_train), np.array(y_train),
                 epochs=1000,batch_size=5,verbose=1,
                 validation_data=(np.array(X_test), np.array(y_test)))
model.save(os.path.join(intents_path, 'intents_chatbot_model.h5'), hist)

print('model created')



Epoch 1/1000
9/9 [==============================] - 1s 22ms/step - loss: 2.4467 - accuracy: 0.1111 - val_loss: 2.3147 - val_accuracy: 0.1875
Epoch 2/1000
9/9 [==============================] - 0s 5ms/step - loss: 2.2187 - accuracy: 0.3333 - val_loss: 2.2391 - val_accuracy: 0.1875
Epoch 3/1000
9/9 [==============================] - 0s 4ms/step - loss: 2.0400 - accuracy: 0.4000 - val_loss: 2.1927 - val_accuracy: 0.1875
Epoch 4/1000
9/9 [==============================] - 0s 4ms/step - loss: 1.9508 - accuracy: 0.3778 - val_loss: 2.1968 - val_accuracy: 0.1875
Epoch 5/1000
9/9 [==============================] - 0s 4ms/step - loss: 1.7885 - accuracy: 0.3778 - val_loss: 2.1512 - val_accuracy: 0.1875
Epoch 6/1000
9/9 [==============================] - 0s 5ms/step - loss: 1.7147 - accuracy: 0.4000 - val_loss: 2.1256 - val_accuracy: 0.1875
Epoch 7/1000
9/9 [==============================] - 0s 4ms/step - loss: 1.5519 - accuracy: 0.5111 - val_loss: 2.0954 - val_accuracy: 0.1875
Epoch 8/1000
9/9 [=